# Semantic Segmentation


In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import sys
import cv2
import scipy
import numpy as np

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## Check for a GPU

In [2]:
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


## Check TensorFlow Version


In [3]:
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.7.0


In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    # Refer https://stackoverflow.com/questions/45705070/how-to-load-and-use-a-saved-model-on-tensorflow
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob   = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out  = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out  = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out  = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

In [5]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """

    std_dev = 0.001
    reg = 0.0001
    
    # 1x1 Convolutions

    conx_1x1_layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1,
                                       padding='SAME',
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer3")

    conx_1x1_layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1,
                                       padding='SAME',
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer4")

    conx_1x1_layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1,
                                       padding='SAME',
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer7")      

    upsample_2x_l7 = tf.layers.conv2d_transpose(vgg_layer7_out, num_classes, 4, strides = (2, 2), padding='SAME',
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_l7")
    
    fuse1 = tf.add(upsample_2x_l7, conx_1x1_layer4)

    upsample_2x_f1 = tf.layers.conv2d_transpose(fuse1, num_classes, 4, strides = (2, 2), padding='SAME',
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_f1")
    
    fuse2 = tf.add(upsample_2x_f1, conx_1x1_layer3)
    
    upsample_2x_f2 = tf.layers.conv2d_transpose(fuse2, num_classes, 16, strides = (8, 8), padding='SAME',
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_f2")
    
    return upsample_2x_f2

In [6]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))

    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss_operation = tf.reduce_mean(cross_entropy)
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_constant = 0.0001
    loss = loss_operation + reg_constant * sum(reg_losses)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    training_operation = optimizer.minimize(loss)
    
    return logits, training_operation, loss

In [7]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    for i in range(epochs):
        for images, labels in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict={input_image : images, 
                                          correct_label : labels,
                                          keep_prob: 0.5,
                                          learning_rate : 0.0001})
            print('Epoch {}/{}; Training Loss:{:.03f}'.format(i+1, epochs, loss))

In [8]:
def gen_test_output_video(sess, logits, keep_prob, image_pl, video_file, image_shape):
    """
    Generate test output using the test images
    :param sess: TF session
    :param logits: TF Tensor for the logits
    :param keep_prob: TF Placeholder for the dropout keep robability
    :param image_pl: TF Placeholder for the image placeholder
    :param image_shape: Tuple - Shape of image
    :return: Output for for each test image
    """
    cap = cv2.VideoCapture(video_file)
    counter=0
    while True:
        ret, frame = cap.read()
        if frame is None:
            break
        image = scipy.misc.imresize(frame, image_shape)

        im_softmax = sess.run(
            [tf.nn.softmax(logits)],
            {keep_prob: 1.0, image_pl: [image]})
        im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
        segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
        mask_full = scipy.misc.imresize(mask, frame.shape)
        mask_full = scipy.misc.toimage(mask_full, mode="RGBA")
        mask = scipy.misc.toimage(mask, mode="RGBA")


        street_im = scipy.misc.toimage(image)
        street_im.paste(mask, box=None, mask=mask)

        street_im_full = scipy.misc.toimage(frame)
        street_im_full.paste(mask_full, box=None, mask=mask_full)

        cv2.imwrite("video_output/video%08d.jpg"%counter,np.array(street_im_full))
        counter=counter+1

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

In [9]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        correct_label = tf.placeholder(dtype=tf.float32, shape=(None, None, None, num_classes), name='correct_label')
        learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)

        outputs = layers(layer3_out, layer4_out, layer7_out, num_classes)
        
        logits, training_operation, loss_operation = optimize(outputs, correct_label, learning_rate, num_classes)

        epochs = 50
        batch_size = 20
        # TODO: Train NN using the train_nn function
        sess.run(tf.global_variables_initializer())
        
        train_nn(sess, epochs, batch_size, get_batches_fn, training_operation, loss_operation, input_image, correct_label, keep_prob, learning_rate)

        saver = tf.train.Saver()
        saver.save(sess, './fcn_ss')
        print("Model saved")

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        # OPTIONAL: Apply the trained model to a video
        
        video_file='project_video.mp4'
        gen_test_output_video(sess, logits, keep_prob, input_image, video_file, image_shape)

In [10]:
run()

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 1/50; Training Loss:0.694
Epoch 1/50; Training Loss:0.700
Epoch 1/50; Training Loss:0.693
Epoch 1/50; Training Loss:0.691
Epoch 1/50; Training Loss:0.687
Epoch 1/50; Training Loss:0.681
Epoch 1/50; Training Loss:0.665
Epoch 1/50; Training Loss:0.632
Epoch 1/50; Training Loss:0.596
Epoch 1/50; Training Loss:0.553
Epoch 1/50; Training Loss:0.492
Epoch 1/50; Training Loss:0.469
Epoch 1/50; Training Loss:0.467
Epoch 1/50; Training Loss:0.477
Epoch 1/50; Training Loss:0.480
Epoch 2/50; Training Loss:0.428
Epoch 2/50; Training Loss:0.415
Epoch 2/50; Training Loss:0.364
Epoch 2/50; Training Loss:0.380
Epoch 2/50; Training Loss:0.407
Epoch 2/50; Training Loss:0.393
Epoch 2/50; Training Loss:0.387
Epoch 2/50; Traini

Epoch 16/50; Training Loss:0.058
Epoch 17/50; Training Loss:0.060
Epoch 17/50; Training Loss:0.050
Epoch 17/50; Training Loss:0.055
Epoch 17/50; Training Loss:0.046
Epoch 17/50; Training Loss:0.051
Epoch 17/50; Training Loss:0.054
Epoch 17/50; Training Loss:0.045
Epoch 17/50; Training Loss:0.043
Epoch 17/50; Training Loss:0.038
Epoch 17/50; Training Loss:0.044
Epoch 17/50; Training Loss:0.046
Epoch 17/50; Training Loss:0.040
Epoch 17/50; Training Loss:0.054
Epoch 17/50; Training Loss:0.054
Epoch 17/50; Training Loss:0.033
Epoch 18/50; Training Loss:0.036
Epoch 18/50; Training Loss:0.045
Epoch 18/50; Training Loss:0.042
Epoch 18/50; Training Loss:0.047
Epoch 18/50; Training Loss:0.054
Epoch 18/50; Training Loss:0.054
Epoch 18/50; Training Loss:0.043
Epoch 18/50; Training Loss:0.047
Epoch 18/50; Training Loss:0.033
Epoch 18/50; Training Loss:0.038
Epoch 18/50; Training Loss:0.048
Epoch 18/50; Training Loss:0.040
Epoch 18/50; Training Loss:0.041
Epoch 18/50; Training Loss:0.043
Epoch 18/5

Epoch 33/50; Training Loss:0.028
Epoch 33/50; Training Loss:0.035
Epoch 33/50; Training Loss:0.032
Epoch 33/50; Training Loss:0.028
Epoch 33/50; Training Loss:0.028
Epoch 33/50; Training Loss:0.028
Epoch 33/50; Training Loss:0.026
Epoch 34/50; Training Loss:0.027
Epoch 34/50; Training Loss:0.032
Epoch 34/50; Training Loss:0.025
Epoch 34/50; Training Loss:0.026
Epoch 34/50; Training Loss:0.027
Epoch 34/50; Training Loss:0.025
Epoch 34/50; Training Loss:0.031
Epoch 34/50; Training Loss:0.034
Epoch 34/50; Training Loss:0.023
Epoch 34/50; Training Loss:0.032
Epoch 34/50; Training Loss:0.034
Epoch 34/50; Training Loss:0.025
Epoch 34/50; Training Loss:0.035
Epoch 34/50; Training Loss:0.032
Epoch 34/50; Training Loss:0.026
Epoch 35/50; Training Loss:0.026
Epoch 35/50; Training Loss:0.031
Epoch 35/50; Training Loss:0.029
Epoch 35/50; Training Loss:0.029
Epoch 35/50; Training Loss:0.030
Epoch 35/50; Training Loss:0.027
Epoch 35/50; Training Loss:0.030
Epoch 35/50; Training Loss:0.031
Epoch 35/5

Epoch 50/50; Training Loss:0.022
Epoch 50/50; Training Loss:0.031
Epoch 50/50; Training Loss:0.021
Epoch 50/50; Training Loss:0.027
Epoch 50/50; Training Loss:0.024
Epoch 50/50; Training Loss:0.026
Epoch 50/50; Training Loss:0.030
Epoch 50/50; Training Loss:0.025
Epoch 50/50; Training Loss:0.022
Epoch 50/50; Training Loss:0.026
Epoch 50/50; Training Loss:0.036
Epoch 50/50; Training Loss:0.033
Epoch 50/50; Training Loss:0.030
Model saved
Training Finished. Saving test images to: ./runs/1532591547.7852333


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:26: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use P

KeyboardInterrupt: 